In [1]:
# Import Packages
import numpy as np
import pandas as pd

# For converting word to vectors
from sklearn.feature_extraction.text import CountVectorizer

# For the model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.


# Data Preparation

In [2]:
data = pd.read_csv('filename')
# Keeping only the neccessary columns
data = data[['text','bias']]

FileNotFoundError: File b'filename' does not exist

In [ ]:
# Only fetch texts that are not labeled irreverent
data = data[data.bias != "index for irreverent"]

# Convert all the text into lowercase and remove punctuations
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print('number of biased text: ', data[ data['text'] == 'index for biased text'].size)
print('number of neutral text: ', data[ data['text'] == 'index for neutral text'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000

# Tokenizer also converts words into vectors
tokenizer = Tokenizer(nb_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

print(type(X))

# LSTM Model using Keras

In [ ]:
# LSTM Network
# hyperparameters: embed_dim, lstm_out, batch_size, dropout_x

embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim, input_length = X.shape[1], dropout=0.2))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

In [ ]:
# Separate train and test dataset
# X is the text, Y is the label

Y = pd.get_dummies(data['text']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

# Training

In [ ]:
# Train the model
batch_size = 32
nb_epoch = 7
model.fit(X_train, Y_train, nb_epoch, batch_size=batch_size, verbose = 2)

# Check the Validation Set

In [ ]:
# Extract a validation set

validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Loss: %.2f" % (score))
print("accuracy of the validation set: %.2f" % (acc))

In [ ]:
# Checking the prediction of the Bias and Neural text accuracy

bias_cnt, neu_cnt, bias_correct, neu_correct = 0, 0, 0, 0

for x in range(len(X_validate)):    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neu_correct += 1
        else:
            bias_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neu_cnt += 1
    else:
        bias_cnt += 1

print("bias_accuracy", bias_correct/bias_cnt*100, "%")
print("neutral_accuracy", neu_correct/neu_cnt*100, "%")

# Predict by sentence if the text is biased